In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/Almabetter/Projects/Stock Price Prediction/GOOG.csv'

In [4]:
df = pd.read_csv(file_path)
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-07-02,2536.790039,2576.959961,2535.379883,2574.379883,2574.379883,1058000
1,2021-07-06,2588.989990,2597.689941,2568.179932,2595.419922,2595.419922,1067500
2,2021-07-07,2606.820068,2612.798096,2595.199951,2601.550049,2601.550049,834000
3,2021-07-08,2565.000000,2600.649902,2560.800049,2583.540039,2583.540039,989000
4,2021-07-09,2578.889893,2596.989990,2578.870117,2591.489990,2591.489990,755300
...,...,...,...,...,...,...,...
247,2022-06-27,2378.699951,2385.000000,2320.014893,2332.449951,2332.449951,1642000
248,2022-06-28,2327.020020,2357.129883,2248.879883,2251.429932,2251.429932,1411600
249,2022-06-29,2242.969971,2273.290039,2231.080078,2245.129883,2245.129883,931400
250,2022-06-30,2209.989990,2226.595947,2146.199951,2187.449951,2187.449951,1902300


In [5]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [6]:
df.shape

(252, 7)

In [7]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-07-02,2536.790039,2576.959961,2535.379883,2574.379883,2574.379883,1058000
1,2021-07-06,2588.989990,2597.689941,2568.179932,2595.419922,2595.419922,1067500
2,2021-07-07,2606.820068,2612.798096,2595.199951,2601.550049,2601.550049,834000
3,2021-07-08,2565.000000,2600.649902,2560.800049,2583.540039,2583.540039,989000
4,2021-07-09,2578.889893,2596.989990,2578.870117,2591.489990,2591.489990,755300


In [8]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,252.000000,252.000000,252.000000,252.000000,252.000000,2.520000e+02
mean,2685.363434,2714.195279,2654.224208,2683.406235,2683.406235,1.321835e+06
std,234.848094,228.665885,236.058349,232.676748,232.676748,5.386850e+05
min,2102.840088,2127.899902,2044.160034,2116.790039,2116.790039,5.933000e+05
25%,2598.942444,2619.908753,2551.157532,2585.010071,2585.010071,9.461000e+05
50%,2742.252076,2768.175049,2714.890014,2740.405029,2740.405029,1.198000e+06
75%,2864.472534,2886.227478,2834.732483,2856.075074,2856.075074,1.552425e+06
max,3037.270020,3042.000000,2997.750000,3014.179932,3014.179932,4.487500e+06


In [9]:
df.corr()

,Open,High,Low,Close,Adj Close,Volume
Open,1.000000,0.994161,0.993127,0.984332,0.984332,-0.315149
High,0.994161,1.000000,0.992898,0.992542,0.992542,-0.294050
Low,0.993127,0.992898,1.000000,0.993700,0.993700,-0.370369
Close,0.984332,0.992542,0.993700,1.000000,1.000000,-0.344281
Adj Close,0.984332,0.992542,0.993700,1.000000,1.000000,-0.344281
Volume,-0.315149,-0.294050,-0.370369,-0.344281,-0.344281,1.000000


In [10]:
figure = go.Figure(data=[go.Candlestick(x=df["Date"], open= df["Open"], high=df["High"], low=df["Low"], close=df["Close"])])


In [11]:

figure.update_layout(title = "Google Stock Price Analysis", xaxis_rangeslider_visible=False)

figure.show()

In [12]:
def prepare_data(df, forecast_col, forecast_out, test_size): 
  label = df[forecast_col].shift(-forecast_out)  #creating new column called label with the last 5 rows are nan
  x = np.array(df[[forecast_col]])  #creating the feature array
  x = preprocessing.scale(x)  #processing the feature array
  x_lately = x[-forecast_out:] #creating the column i want to use later in the predicting method  
  x = x[:-forecast_out]  # X that will contain the training and testing
  label.dropna(inplace=True) #dropping na values
  y = np.array(label)  # assigning Y

  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0) #cross validation

  response = [x_train,x_test, y_train, y_test, x_lately]

  return response

In [13]:
forecast_col = 'Close'
forecast_out = 5
test_size = 0.2

In [25]:
x_train, x_test, y_train, y_test, x_lately = prepare_data(df, forecast_col, forecast_out, test_size);  #calling the method were the cross validation and data preperation is in
learner = LinearRegression()  #initializing linear regression model

learner.fit(x_train, y_train)  #training the linear regression model

LinearRegression()

In [15]:
learner.intercept_

2684.6155830536977

In [16]:
learner.coef_

array([216.54306773])

In [36]:
y_pred_train = learner.predict(x_train)

In [17]:
score = learner.score(x_test,y_test) #testing the linear regression model
forecast = learner.predict(x_lately) #set that will contain the forecasted data
response = {}  #creting json object
response['test_score'] = score
response['forecast_set'] = forecast

response

{'forecast_set': array([2357.34442537, 2281.79223672, 2275.91736164, 2222.13009823,
        2216.69370456]), 'test_score': 0.7766966936035584}

In [28]:
y_test

array([2987.76001 , 2899.409912, 2775.459961, 2947.370117, 2848.300049,
       2832.959961, 2865.      , 2582.419922, 2609.350098, 2682.600098,
       2143.879883, 2388.22998 , 2753.070068, 2868.120117, 2300.409912,
       2729.300049, 2814.      , 2916.840088, 2793.439941, 2801.120117,
       2291.280029, 2263.219971, 2653.469971, 2888.330078, 2869.300049,
       2567.48999 , 2973.5     , 2942.850098, 2666.570068, 2584.800049,
       2856.120117, 2751.02002 , 2343.139893, 2585.080078, 2753.790039,
       2901.48999 , 2727.629883, 2340.209961, 2778.76001 , 2962.120117,
       2534.709961, 2646.169922, 2723.679932, 2261.679932, 2697.820068,
       2934.350098, 2214.909912, 2875.530029, 2776.949951, 2738.800049])

In [31]:
y_pred = learner.predict(x_test)
y_pred

array([2967.74859645, 2943.22346935, 2848.55477239, 2955.98017833,
       2754.17520841, 2749.5779847 , 2798.51622828, 2672.23521706,
       2683.86384968, 2849.58996115, 2368.6652561 , 2512.4213166 ,
       2914.65138397, 2896.27136944, 2574.11626822, 2786.80381499,
       2821.71703269, 2848.35898111, 2864.62194585, 2727.36547113,
       2202.05307488, 2359.6570389 , 2746.48197945, 2913.5975263 ,
       2882.39569104, 2813.16596487, 2840.3486158 , 2883.5800007 ,
       2630.45883437, 2712.24947824, 2993.06622312, 2905.28892109,
       2480.94901632, 2617.35698664, 2719.27132516, 2943.73639698,
       2655.33820716, 2286.03521381, 2713.11665154, 2863.77344144,
       2724.1576828 , 2767.27705615, 2786.74781015, 2367.31293099,
       2615.55730166, 2962.34931213, 2292.78659665, 2918.62389698,
       2677.056462  , 2728.82024581])

In [38]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from numpy import math

In [34]:
mean_squared_error(y_test, y_pred)

11212.456625640016

In [39]:
math.sqrt(mean_squared_error(y_train, y_pred_train))

102.97725306279504

In [40]:
r2_score(y_train, y_pred_train)

0.8105845626067134

In [41]:
r2_score(y_test, y_pred)

0.7766966936035584

# Now let's have predict the stock prices for next 5 days

In [ ]:
pip install autots

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 486 kB 5.2 MB/s 


In [ ]:
from autots import AutoTS


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
model = AutoTS(forecast_length=5, frequency='infer', ensemble='simple')

In [ ]:
model = model.fit(df, date_col = 'Date', value_col='Close',id_col = None)


Inferred frequency is: None
Frequency is 'None'! Input frequency not recognized.
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Model Number: 6 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Model Number: 7 with model DatepartRegression in generation 0 of 10
Epoch 1/50
12/12 [==============================] - 12s 11ms/step - loss: 0.4042
Epoch 2/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3897
Epoch 3/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3762
Epoch 4/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3582
Epoch 5/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3463
Epoch 6/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3331
Epoch 7/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3224
Epoch 8/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3184
Epoch 9/50
12/12 [==============================] - 0s 10ms/step - loss: 0.3118
Epoch 10/50
12/12 [==============================] - 0s 10ms/step - loss: 0.3040
Epoch 11/50
12/12 [==============================] - 0s 10ms/step - loss: 0.2971
Epoch 12/50
12/12 [==============================] - 0s 

...

142 - AverageValueNaive with avg smape 3.78: 
Model Number: 143 of 159 with model UnivariateRegression for Validation 3
143 - UnivariateRegression with avg smape 9.42: 
Model Number: 144 of 159 with model GLM for Validation 3


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


144 - GLM with avg smape 16.65: 
Model Number: 145 of 159 with model UnivariateRegression for Validation 3
145 - UnivariateRegression with avg smape 5.37: 
Model Number: 146 of 159 with model UnobservedComponents for Validation 3
146 - UnobservedComponents with avg smape 3.18: 
Model Number: 147 of 159 with model UnobservedComponents for Validation 3
147 - UnobservedComponents with avg smape 15.16: 
Model Number: 148 of 159 with model UnivariateRegression for Validation 3


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


148 - UnivariateRegression with avg smape 12.95: 
Model Number: 149 of 159 with model UnobservedComponents for Validation 3
149 - UnobservedComponents with avg smape 16.22: 
Model Number: 150 of 159 with model UnivariateRegression for Validation 3
150 - UnivariateRegression with avg smape 3.5: 
Model Number: 151 of 159 with model UnivariateRegression for Validation 3
151 - UnivariateRegression with avg smape 7.96: 
Model Number: 152 of 159 with model UnobservedComponents for Validation 3
152 - UnobservedComponents with avg smape 16.57: 
Model Number: 153 of 159 with model UnobservedComponents for Validation 3
153 - UnobservedComponents with avg smape 15.98: 
Model Number: 154 of 159 with model UnobservedComponents for Validation 3
154 - UnobservedComponents with avg smape 16.08: 
Model Number: 155 of 159 with model UnivariateRegression for Validation 3
155 - UnivariateRegression with avg smape 21.17: 
Model Number: 156 of 159 with model GLM for Validation 3
156 - GLM with avg smape 22.

In [ ]:
prediction = model.predict() 

In [ ]:
forecast = prediction.forecast  

In [ ]:
forecast

,Close
2022-07-02,2245.194776
2022-07-03,2220.135464
2022-07-04,2208.075328
2022-07-05,2212.325117
2022-07-06,2216.334465
